In [51]:
import contextlib

In [52]:
run = print

In [71]:
def stop_database():
    run("systemctl stop postgresql.service")


def start_database():
    run("systemctl start postgresql.service")
    
    
class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        print(exc_type)
        print(type(exc_type))
        print(ex_value)
        print(type(ex_value))
        print(ex_traceback)
        print(type(ex_traceback))
        start_database()


@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

    
def db_backup():
    run("pg_dump database")

    
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()


@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

In [72]:
def main():
    with DBHandler():
        db_backup()

    with db_handler():
        db_backup()

    offline_backup()


In [75]:
with DBHandler():
    db_backup()
    raise

systemctl stop postgresql.service
pg_dump database
<class 'RuntimeError'>
<class 'type'>
No active exception to reraise
<class 'RuntimeError'>
<class 'traceback'>
systemctl start postgresql.service


RuntimeError: No active exception to reraise

In [77]:
with db_handler():
    db_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [6]:
offline_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [18]:
from datetime import timedelta
from datetime import date


class DateRangeIterable:
    """An iterable that contains its own iterator object."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today


class DateRangeContainerIterable:
    """An range that builds its iteration through a generator."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date

    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)


class DateRangeSequence:
    """An range created by wrapping a sequence."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()

    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days

    def __getitem__(self, day_no):
        return self._range[day_no]

    def __len__(self):
        return len(self._range)


In [72]:
s1 = DateRangeIterable(date(2020, 1, 1), date(2020, 1, 5))

In [73]:
for i in s1:
    print(i)

2020-01-01
2020-01-02
2020-01-03
2020-01-04


In [74]:
s1[1]

TypeError: 'DateRangeIterable' object does not support indexing

In [75]:
s1 = DateRangeContainerIterable(date(2020, 1, 1), date(2020, 1, 5))

In [76]:
for i in s1:
    print(i)

2020-01-01
2020-01-02
2020-01-03
2020-01-04


In [78]:
len(s1)

TypeError: object of type 'DateRangeContainerIterable' has no len()

In [79]:
s1 = DateRangeSequence(date(2020, 1, 1), date(2020, 1, 5))

In [80]:
len(s1)

4

In [81]:
s1[0]

datetime.date(2020, 1, 1)

In [82]:
for i in s1:
    print(i)

2020-01-01
2020-01-02
2020-01-03
2020-01-04


In [32]:
from collections import UserList

In [33]:
def wrong_user_display(user_metadata: dict = {"name": "John", "age": 30}):
    
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")

    return f"{name} ({age})"

In [34]:
inputs = {
    'name': 'John',
    'age': 20
}

In [35]:
wrong_user_display(inputs)

'John (20)'

In [36]:
inputs

{}

In [38]:
wrong_user_display()

KeyError: 'name'

In [25]:
def wrong_user_display_v2(user_metadata: dict = {"name": "John", "age": 30}):
    
    user_metadata = user_metadata.copy()
    
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")

    return f"{name} ({age})"

In [26]:
inputs = {
    'name': 'John',
    'age': 20
}

In [27]:
wrong_user_display_v2(inputs)

'John (20)'

In [28]:
inputs

{'name': 'John', 'age': 20}

In [39]:
def user_display(user_metadata: dict = None):
    user_metadata = user_metadata or {"name": "John", "age": 30}

    name = user_metadata.pop("name")
    age = user_metadata.pop("age")

    return f"{name} ({age})"

In [29]:
inputs = {
    'name': 'John',
    'age': 20
}

In [30]:
user_display(inputs)

'John (20)'

In [31]:
inputs

{}

In [44]:
from collections import UserList, UserDict, UserString, 

In [45]:
list

list

In [56]:
class BadList(list):
    def __getitem__(self, index):
        print('===')
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}"


class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "even"
        else:
            prefix = "odd"
        return f"[{prefix}] {value}"

In [57]:
bad_list = BadList([1,2,3])

In [58]:
bad_list

[1, 2, 3]

In [59]:
for i in bad_list:
    print(i)

1
2
3


In [67]:
bad_list[1:3]

===


TypeError: unsupported operand type(s) for %: 'slice' and 'int'

In [68]:
bad_list.__getitem__(0)

===


'[even] 1'

In [50]:
good_list = GoodList([1, 2, 3])

In [51]:
for i in good_list:
    print(i)

[even] 1
[odd] 2
[even] 3


In [70]:
good_list[1:3]

TypeError: unsupported operand type(s) for %: 'slice' and 'int'

In [74]:
list.__mro__

(list, object)

In [78]:
BadList.__mro__

(__main__.BadList, list, object)

In [79]:
UserList.__mro__

(collections.UserList,
 collections.abc.MutableSequence,
 collections.abc.Sequence,
 collections.abc.Reversible,
 collections.abc.Collection,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object)

In [77]:
GoodList.__mro__

(__main__.GoodList,
 collections.UserList,
 collections.abc.MutableSequence,
 collections.abc.Sequence,
 collections.abc.Reversible,
 collections.abc.Collection,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object)

In [22]:
a = [1, 2, 3, 4, 5]

In [23]:
b = a.copy()

In [24]:
b[3] = 'a'

In [25]:
b

[1, 2, 3, 'a', 5]

In [26]:
a

[1, 2, 3, 4, 5]

In [30]:
'asdfasdfasdf'[1:10:2]

'sfsfs'

In [116]:
class Person:    
    def __init__(self):
        self._age = 0
        self.__gender = 'M'
    
    @property
    def age(self):
        return self._age
    
    
    def __private(self):
        print(1)

In [117]:
p = Person()

In [118]:
p.__init__()

In [128]:
p._Person__private()

1


In [127]:
p.__dict__

{'_age': 0, '_Person__gender': 'M'}

In [81]:
p.age

0

In [82]:
p.age = 1

AttributeError: can't set attribute

In [83]:
p.__dict__

{'_age': 0}

In [84]:
p.__dict__['_age'] = 1

In [86]:
p.age

1

In [90]:
list.__dict__

mappingproxy({'__repr__': <slot wrapper '__repr__' of 'list' objects>,
              '__hash__': None,
              '__getattribute__': <slot wrapper '__getattribute__' of 'list' objects>,
              '__lt__': <slot wrapper '__lt__' of 'list' objects>,
              '__le__': <slot wrapper '__le__' of 'list' objects>,
              '__eq__': <slot wrapper '__eq__' of 'list' objects>,
              '__ne__': <slot wrapper '__ne__' of 'list' objects>,
              '__gt__': <slot wrapper '__gt__' of 'list' objects>,
              '__ge__': <slot wrapper '__ge__' of 'list' objects>,
              '__iter__': <slot wrapper '__iter__' of 'list' objects>,
              '__init__': <slot wrapper '__init__' of 'list' objects>,
              '__len__': <slot wrapper '__len__' of 'list' objects>,
              '__getitem__': <method '__getitem__' of 'list' objects>,
              '__setitem__': <slot wrapper '__setitem__' of 'list' objects>,
              '__delitem__': <slot wrapper '__del

In [138]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method


class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

In [143]:
mapping = Mapping([1,2,3,4])

In [144]:
submapping = MappingSubclass([1,2,3,4,5])

In [146]:
submapping.items_list

[1, 2, 3, 4, 5]

In [147]:
submapping.update([1], [3])

In [148]:
submapping.items_list

[1, 2, 3, 4, 5, (1, 3)]

In [150]:
submapping._Mapping__update([1,2,3])

In [151]:
submapping.items_list

[1, 2, 3, 4, 5, (1, 3), 1, 2, 3]